In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv("small_training.csv")

In [3]:
df = pd.read_csv("AbstractAItraining.csv")

In [4]:
df.head()

,prompt,responses,text
0,1. Cloud computing and energy consumption: Clo...,Cloud computing offers utilityoriented IT serv...,### Instruction: 1. Cloud computing and energy...
1,1. Liposomes were first described in the 1960s...,Liposomes sphereshaped vesicles consisting of ...,### Instruction: 1. Liposomes were first descr...
2,1. Big Data Attracting Attention: The expandin...,It is already true that Big Data has drawn hug...,### Instruction: 1. Big Data Attracting Attent...
3,1. Propensity Score: This is the probability o...,In a study comparing the effects of two treatm...,### Instruction: 1. Propensity Score: This is ...
4,1. Context for Socially Interactive Robots: Th...,This paper reviews socially interactive robots...,### Instruction: 1. Context for Socially Inter...


In [7]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
# model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
# dataset_name = "small_training.csv"
# new_model = "llama-2-7b-custom"
# lora_r = 64
# lora_alpha = 32
# lora_dropout = 0.05
# use_4bit = True
# bnb_4bit_compute_dtype = "float16"
# bnb_4bit_quant_type = "nf4"
# use_nested_quant = False
# output_dir = "./results"
# num_train_epochs = 1
# fp16 = False
# bf16 = False
# # per_device_train_batch_size = 4
# per_device_eval_batch_size = 4
# gradient_accumulation_steps = 1
# gradient_checkpointing = True
# max_grad_norm = 0.3
# learning_rate = 2e-4
# weight_decay = 0.001
# optim = "paged_adamw_32bit"
# lr_scheduler_type = "constant"
# max_steps = -1
# warmup_ratio = 0.03
# group_by_length = True
# save_steps = 25
# logging_steps = 5
# max_seq_length = None
# packing = False
# device_map = {"": 0}

In [6]:
# Load datasets from CSV
train_dataset = load_dataset('csv', data_files='small_training.csv', split="train")
valid_dataset = load_dataset('csv', data_files='small_training.csv', split="train")

#Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [prompt + ' ' + response for prompt, response in zip(examples['prompt'], examples['responses'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [prompt + ' ' + response for prompt, response in zip(examples['prompt'], examples['responses'])]}, batched=True)


In [ ]:
# train_dataset = load_dataset('csv', data_files='small_training.csv', split="train")
#  valid_dataset = load_dataset('csv', data_files='small_training.csv', split="train")

In [ ]:
# train_dataset = load_dataset('csv', data_files='AbstractAItraining.csv', split="train")
# valid_dataset = load_dataset('csv', data_files='AbstractAItraining.csv', split="train")

In [8]:
valid_dataset_mapped

Dataset({
    features: ['prompt', 'responses', 'text'],
    num_rows: 326
})

In [ ]:
# model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument

# lora_r = 64
# lora_alpha = 32
# lora_dropout = 0.05

# output_dir = "./results"

# per_device_train_batch_size = 4

# gradient_accumulation_steps = 1

# max_seq_length = None

# device_map = {"": 0}

### BNB

In [9]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= "float16", 
    bnb_4bit_use_double_quant= False,
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    load_in_4bit=True,
    device_map={"": 0}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:363: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:368: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [11]:
model.config.use_cache = False ### 
model.config.pretraining_tp = 1 ###

In [12]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
adapters_name = 'timdettmers/guanaco-7b'

In [ ]:
#model = PeftModel.from_pretrained(model , adapters_name)

### PEFT

In [14]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
model = prepare_model_for_kbit_training(model)
#model = PeftModel.from_pretrained(model , adapters_name)
model = get_peft_model(model, peft_config=peft_config)

### Trainable Parameters 

In [15]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
print_trainable_parameters(model)

trainable params: 33554432 || all params: 3533967360 || trainable%: 0.9494833591219133


In [17]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=1,
    optim= "paged_adamw_32bit",
    save_steps= 25,
    logging_steps=5,
    warmup_ratio= 0.03,
    learning_rate=2e-4,
    weight_decay=  0.001,
    fp16=False,
    bf16=False,
    max_grad_norm= 0.3,
    max_steps= -1,
    group_by_length= True,
    lr_scheduler_type="constant",
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  
)


In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 326/326 [00:00<00:00, 1581.84 examples/s]


Map: 100%|██████████| 326/326 [00:00<00:00, 2539.85 examples/s]
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [20]:
trainer.train()
trainer.model.save_pretrained(new_model) 

Step,Training Loss,Validation Loss
5,2.026200,2.004630
10,1.940000,1.898936
15,1.887900,1.830923


In [ ]:
prompt = f"[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as Multalign Viewer, ViewDock, Movie, and Volume Viewer offer a diverse set of features. They allow the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data respectively.\n5. Real-World Usage of Chimera: The abstract also discusses the practical usage of Chimera in real-world situations, pointing out its wide applicability and impact in the field of molecular biology and bioinformatics \n . [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(prompt)
print(result)

In [ ]:
# #Cell 4: Test the model
# logging.set_verbosity(logging.CRITICAL)
# prompt = f"[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as Multalign Viewer, ViewDock, Movie, and Volume Viewer offer a diverse set of features. They allow the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data respectively.\n5. Real-World Usage of Chimera: The abstract also discusses the practical usage of Chimera in real-world situations, pointing out its wide applicability and impact in the field of molecular biology and bioinformatics \n . [/INST]" # replace the command here with something relevant to your task

# result = pipe(prompt) 

In [ ]:
# print(result[0]) 

{'generated_text': '[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as 